# SSD MVTec Anomaly Detection Analysis

This notebook provides comprehensive analysis of the SSD model trained on MVTec dataset, including:
- Dataset statistics
- Training configuration and results
- Model performance metrics
- Qualitative inspection with visualizations


In [ ]:
%matplotlib inline

from pathlib import Path
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision.models.detection.ssd import SSDClassificationHead
from PIL import Image
import numpy as np
import cv2

sns.set_theme(style="whitegrid")


In [ ]:
# Configuration - UPDATE THESE PATHS
BASE_DIR = Path("/Users/vantrinh/Work/python/yolo")
DATASET_DIR = BASE_DIR / "datasets/ssd_mvtec_bottle"
MODEL_DIR = BASE_DIR / "model_ssd/ssd_mvtec_20251120_215313"  # UPDATE THIS
MODEL_PATH = MODEL_DIR / "best.pt"
INFERENCE_DIR = MODEL_DIR / "inference_val"


## 1. Dataset Overview

Compare dataset sizes before and after augmentation.


In [ ]:
# Check if we're using augmented dataset
augmented_dataset_dir = BASE_DIR / "datasets/ssd_mvtec_bottle_augmented"
original_dataset_dir = BASE_DIR / "datasets/ssd_mvtec_bottle"

# Determine which dataset was used for training
if DATASET_DIR.exists() and "augmented" in str(DATASET_DIR):
    using_augmented = True
    aug_info_path = DATASET_DIR / "augmentation_info.json"
else:
    using_augmented = False
    # Check if augmented dataset exists even if not used
    if augmented_dataset_dir.exists():
        aug_info_path = augmented_dataset_dir / "augmentation_info.json"
    else:
        aug_info_path = None

print("📊 DATASET COMPARISON")
print("="*60)

# Load original dataset info
orig_info_path = original_dataset_dir / "dataset_info.json"
if orig_info_path.exists():
    with open(orig_info_path, 'r') as f:
        orig_info = json.load(f)
    
    print("\n✓ Original Dataset:")
    print(f"   Train images: {orig_info.get('train_images', 0)}")
    print(f"   Val images:   {orig_info.get('val_images', 0)}")
    print(f"   Total:        {orig_info.get('train_images', 0) + orig_info.get('val_images', 0)}")
else:
    print("\n⚠️  Original dataset info not found")
    orig_info = None

# Load augmented dataset info
if aug_info_path and aug_info_path.exists():
    with open(aug_info_path, 'r') as f:
        aug_info = json.load(f)
    
    stats = aug_info.get('stats', {})
    train_stats = stats.get('train', {})
    val_stats = stats.get('val', {})
    
    print(f"\n✓ Augmented Dataset (factor: {aug_info.get('augmentation_factor', 'N/A')}):")
    print(f"   Train images: {train_stats.get('total', 0)} (original: {train_stats.get('original', 0)}, augmented: {train_stats.get('augmented', 0)})")
    print(f"   Val images:   {val_stats.get('total', 0)} (original: {val_stats.get('original', 0)}, augmented: {val_stats.get('augmented', 0)})")
    print(f"   Total:        {train_stats.get('total', 0) + val_stats.get('total', 0)}")
    
    print(f"\n✓ Used for training: {'Augmented' if using_augmented else 'Original'} dataset")
    
    # Create comparison DataFrame
    comparison_data = []
    
    if orig_info:
        comparison_data.append({
            "Dataset": "Original",
            "Train": orig_info.get('train_images', 0),
            "Val": orig_info.get('val_images', 0),
            "Total": orig_info.get('train_images', 0) + orig_info.get('val_images', 0)
        })
    
    comparison_data.append({
        "Dataset": "Augmented",
        "Train": train_stats.get('total', 0),
        "Val": val_stats.get('total', 0),
        "Total": train_stats.get('total', 0) + val_stats.get('total', 0)
    })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n")
    display(comparison_df)
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart comparison
    x = np.arange(len(comparison_data))
    width = 0.25
    
    axes[0].bar(x - width, [d['Train'] for d in comparison_data], width, label='Train', color='#2E86AB')
    axes[0].bar(x, [d['Val'] for d in comparison_data], width, label='Val', color='#A23B72')
    axes[0].bar(x + width, [d['Total'] for d in comparison_data], width, label='Total', color='#F18F01')
    
    axes[0].set_ylabel('Number of Images')
    axes[0].set_title('Dataset Comparison: Before and After Augmentation')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels([d['Dataset'] for d in comparison_data])
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Augmentation breakdown
    if orig_info:
        train_increase = ((train_stats.get('total', 0) - orig_info.get('train_images', 0)) / orig_info.get('train_images', 1)) * 100
        val_increase = ((val_stats.get('total', 0) - orig_info.get('val_images', 0)) / orig_info.get('val_images', 1)) * 100
        
        axes[1].bar(['Train', 'Val'], [train_increase, val_increase], color=['#2E86AB', '#A23B72'])
        axes[1].set_ylabel('Increase (%)')
        axes[1].set_title('Dataset Growth After Augmentation')
        axes[1].grid(axis='y', alpha=0.3)
        
        for i, (label, val) in enumerate(zip(['Train', 'Val'], [train_increase, val_increase])):
            axes[1].text(i, val + 5, f'{val:.1f}%', ha='center', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("\n⚠️  Augmented dataset not found")
    print("Run augmentation first:")
    print(f"python augment_ssd_dataset.py --dataset_dir {original_dataset_dir}")

print("="*60)


## 2. Model Evaluation

Analyze the model's performance on the validation set.


In [ ]:
# Load evaluation metrics
metrics_path = INFERENCE_DIR / "metrics.json"

if metrics_path.exists():
    with open(metrics_path, 'r') as f:
        metrics = json.load(f)
    
    print("✓ Evaluation metrics loaded\n")
    
    # Display overall metrics
    print("="*60)
    print("OVERALL METRICS")
    print("="*60)
    print(f"Accuracy : {metrics.get('accuracy', 0)*100:>6.2f}%")
    print(f"Precision: {metrics.get('precision', 0)*100:>6.2f}%")
    print(f"Recall   : {metrics.get('recall', 0)*100:>6.2f}%")
    print(f"F1-Score : {metrics.get('f1_score', 0)*100:>6.2f}%")
    print(f"Defect detection rate: {metrics.get('defect_detection_rate', 0)*100:>6.2f}%")
    print(f"Composite: {metrics.get('composite_score', 0)*100:>6.2f}%")
    print(f"Confidence: {metrics.get('confidence', 0)*100:>6.2f}%")
    print("="*60)
    
    # Model Scorecard
    scorecard_df = pd.DataFrame([
        {"Metric": "Accuracy", "Value": f"{metrics.get('accuracy', 0)*100:.2f}%"},
        {"Metric": "Precision", "Value": f"{metrics.get('precision', 0)*100:.2f}%"},
        {"Metric": "Recall", "Value": f"{metrics.get('recall', 0)*100:.2f}%"},
        {"Metric": "F1-Score", "Value": f"{metrics.get('f1_score', 0)*100:.2f}%"},
        {"Metric": "Defect detection rate", "Value": f"{metrics.get('defect_detection_rate', 0)*100:.2f}%"},
        {"Metric": "Composite Score", "Value": f"{metrics.get('composite_score', 0)*100:.2f}%"},
        {"Metric": "Confidence", "Value": f"{metrics.get('confidence', 0)*100:.2f}%"},
    ])
    
    print("\n📊 Model Scorecard:")
    display(scorecard_df)
    
    # Confusion Matrix
    confusion_df = pd.DataFrame([
        {"Metric": "TP (Defect detected)", "Count": metrics.get('tp_defect', 0)},
        {"Metric": "FP (False defect)", "Count": metrics.get('fp_defect', 0)},
        {"Metric": "TN (Good correct)", "Count": metrics.get('tn_good', 0)},
        {"Metric": "FN (Defect missed)", "Count": metrics.get('fn_defect', 0)},
        {"Metric": "Total images", "Count": metrics.get('total_images', 0)},
    ])
    
    print("\n📈 Confusion Matrix:")
    display(confusion_df)
    
else:
    print(f"⚠️ Metrics not found at {metrics_path}")
    print("Please run inference first:")


## 3. Sample Image Inspection

Test the model on sample images (2 good + 2 defect) and visualize predictions.


In [ ]:
# Helper functions for SSD inference
def load_ssd_model(model_path, num_classes=2, device='cpu'):
    """Load trained SSD model."""
    in_channels = [512, 1024, 512, 256, 256, 256]
    num_anchors = [4, 6, 6, 6, 4, 4]
    
    model = ssd300_vgg16(num_classes=num_classes + 1)
    model.head.classification_head = SSDClassificationHead(
        in_channels=in_channels,
        num_anchors=num_anchors,
        num_classes=num_classes + 1
    )
    
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    return model

def run_inference(model, image, device, conf_threshold=0.5):
    """Run inference on image array."""
    # Convert to RGB if needed
    if len(image.shape) == 2:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Convert to tensor
    image_tensor = torch.from_numpy(image_rgb).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)
    
    # Run inference
    with torch.no_grad():
        predictions = model(image_tensor)
    
    # Extract predictions
    pred = predictions[0]
    boxes = pred['boxes'].cpu().numpy()
    labels = pred['labels'].cpu().numpy()
    scores = pred['scores'].cpu().numpy()
    
    # Filter by confidence threshold
    mask = scores >= conf_threshold
    boxes = boxes[mask]
    labels = labels[mask]
    scores = scores[mask]
    
    return boxes, labels, scores

def draw_predictions(image, boxes, labels, scores, class_names=['background', 'good', 'defect']):
    """Draw bounding boxes on image."""
    image = image.copy()
    
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = map(int, box)
        
        color = (0, 255, 0) if label == 1 else (0, 0, 255)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        
        class_name = class_names[label] if label < len(class_names) else f'class_{label}'
        text = f"{class_name}: {score:.2f}"
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def get_verdict(boxes, labels):
    """Get verdict: good or defect."""
    if len(boxes) == 0:
        return "GOOD"
    has_defect = 2 in labels
    return "DEFECT" if has_defect else "GOOD"

print("✓ Helper functions defined")


In [ ]:
# Load model
device = torch.device('mps' if torch.backends.mps.is_available() else 
                      'cuda' if torch.cuda.is_available() else 'cpu')

print(f"Loading model from {MODEL_PATH}...")
print(f"Device: {device}")

model = load_ssd_model(MODEL_PATH, num_classes=2, device=device)
print("✓ Model loaded successfully")


In [ ]:
# Select sample images (2 good + 2 defect)
val_images_dir = DATASET_DIR / "val" / "images"
val_ann_file = DATASET_DIR / "val" / "annotations.json"

# Load annotations to find good and defect images
with open(val_ann_file, 'r') as f:
    val_data = json.load(f)

# Create mapping of image_id to annotations
img_to_anns = {}
for ann in val_data['annotations']:
    img_id = ann['image_id']
    if img_id not in img_to_anns:
        img_to_anns[img_id] = []
    img_to_anns[img_id].append(ann)

# Find good and defect images
good_images = []
defect_images = []

for img_info in val_data['images']:
    img_id = img_info['id']
    anns = img_to_anns.get(img_id, [])
    
    # Check if has defect (category_id == 2)
    has_defect = any(ann['category_id'] == 2 for ann in anns)
    
    if has_defect:
        defect_images.append(img_info)
    else:
        good_images.append(img_info)

# Select 2 of each
selected_defect = defect_images[:2] if len(defect_images) >= 2 else defect_images
selected_good = good_images[:2] if len(good_images) >= 2 else good_images

print(f"Found {len(defect_images)} defect images, {len(good_images)} good images")
print(f"Selected 2 defect + 2 good images for testing")
print(f"\nDefect images: {[img['file_name'] for img in selected_defect]}")
print(f"Good images: {[img['file_name'] for img in selected_good]}")


In [ ]:
# Visualize original images with predictions
sample_images = selected_defect + selected_good
sample_labels = ['Defect'] * len(selected_defect) + ['Good'] * len(selected_good)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

print("\nRunning inference on sample images...")

for idx, (img_info, gt_label) in enumerate(zip(sample_images, sample_labels)):
    img_path = val_images_dir / img_info['file_name']
    
    # Load image
    image = cv2.imread(str(img_path))
    
    # Run inference
    boxes, labels, scores = run_inference(model, image, device, conf_threshold=0.5)
    
    # Draw predictions
    annotated_image = draw_predictions(image, boxes, labels, scores)
    
    # Get verdict
    verdict = get_verdict(boxes, labels)
    
    # Display
    axes[idx].imshow(annotated_image)
    
    # Create title
    if len(boxes) > 0:
        avg_conf = np.mean(scores)
        title = f"{img_info['file_name']}\nGT: {gt_label} | Pred: {verdict} (conf: {avg_conf:.2f})"
        match = "✓" if gt_label.upper() == verdict else "✗"
    else:
        title = f"{img_info['file_name']}\nGT: {gt_label} | Pred: {verdict} (no detections)"
        match = "✓" if gt_label.upper() == verdict else "✗"
    
    title += f" {match}"
    color = 'green' if match == "✓" else 'red'
    
    axes[idx].set_title(title, fontsize=10, color=color, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\n✓ Original images displayed")


## 4. Rotation Test

Test model robustness by rotating images (0°, 90°, 180°, 270°) and checking if predictions remain consistent.


In [ ]:
# Test rotation on first image of each type
test_images = [selected_defect[0], selected_good[0]]
test_labels = ['Defect', 'Good']

rotation_angles = {
    '0°': None,
    '90°': cv2.ROTATE_90_CLOCKWISE,
    '180°': cv2.ROTATE_180,
    '270°': cv2.ROTATE_90_COUNTERCLOCKWISE
}

print("Testing rotation robustness...\n")

for img_info, gt_label in zip(test_images, test_labels):
    img_path = val_images_dir / img_info['file_name']
    original_image = cv2.imread(str(img_path))
    
    print(f"\n{'='*60}")
    print(f"Testing: {img_info['file_name']} (GT: {gt_label})")
    print(f"{'='*60}")
    
    # Create subplot for this image
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    results_summary = []
    
    for idx, (angle, rotation_code) in enumerate(rotation_angles.items()):
        # Rotate image
        if rotation_code is None:
            rotated_image = original_image.copy()
        else:
            rotated_image = cv2.rotate(original_image, rotation_code)
        
        # Run inference
        boxes, labels, scores = run_inference(model, rotated_image, device, conf_threshold=0.5)
        
        # Draw predictions
        annotated_image = draw_predictions(rotated_image, boxes, labels, scores)
        
        # Get verdict
        verdict = get_verdict(boxes, labels)
        match = "✓" if gt_label.upper() == verdict else "✗"
        
        # Display
        axes[idx].imshow(annotated_image)
        
        # Create title
        if len(boxes) > 0:
            avg_conf = np.mean(scores)
            title = f"Rotation: {angle}\nPred: {verdict} (conf: {avg_conf:.2f}) {match}"
        else:
            title = f"Rotation: {angle}\nPred: {verdict} (no detections) {match}"
        
        color = 'green' if match == "✓" else 'red'
        axes[idx].set_title(title, fontsize=9, color=color, fontweight='bold')
        axes[idx].axis('off')
        
        # Store result
        results_summary.append({
            'Rotation': angle,
            'Prediction': verdict,
            'Detections': len(boxes),
            'Avg Confidence': f"{np.mean(scores):.2f}" if len(scores) > 0 else "N/A",
            'Match': match
        })
        
        print(f"  {angle:>4s}: Pred={verdict:>6s}, Detections={len(boxes)}, Match={match}")
    
    plt.suptitle(f"{img_info['file_name']} - GT: {gt_label}", fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Display summary table
    summary_df = pd.DataFrame(results_summary)
    print("\nRotation Test Summary:")
    display(summary_df)

print("\n" + "="*60)
print("✓ Rotation test completed")
print("="*60)


## 5. Complete Rotation Matrix

Test all 4 sample images with all rotation angles in a comprehensive view.


In [ ]:
# Create comprehensive rotation matrix for all 4 images
print("Creating complete rotation matrix...\n")

# 4 images x 4 rotations = 16 subplots
fig, axes = plt.subplots(4, 4, figsize=(16, 16))

overall_results = []

for row_idx, (img_info, gt_label) in enumerate(zip(sample_images, sample_labels)):
    img_path = val_images_dir / img_info['file_name']
    original_image = cv2.imread(str(img_path))
    
    for col_idx, (angle, rotation_code) in enumerate(rotation_angles.items()):
        # Rotate image
        if rotation_code is None:
            rotated_image = original_image.copy()
        else:
            rotated_image = cv2.rotate(original_image, rotation_code)
        
        # Run inference
        boxes, labels, scores = run_inference(model, rotated_image, device, conf_threshold=0.5)
        
        # Draw predictions
        annotated_image = draw_predictions(rotated_image, boxes, labels, scores)
        
        # Get verdict
        verdict = get_verdict(boxes, labels)
        match = "✓" if gt_label.upper() == verdict else "✗"
        
        # Display
        ax = axes[row_idx, col_idx]
        ax.imshow(annotated_image)
        
        # Create title
        if col_idx == 0:
            # First column - show image name and GT
            title = f"{img_info['file_name'][:15]}...\nGT: {gt_label} | {angle}"
        else:
            title = angle
        
        if len(boxes) > 0:
            avg_conf = np.mean(scores)
            title += f"\n{verdict} ({avg_conf:.2f}) {match}"
        else:
            title += f"\n{verdict} {match}"
        
        color = 'green' if match == "✓" else 'red'
        ax.set_title(title, fontsize=8, color=color, fontweight='bold')
        ax.axis('off')
        
        # Store result
        overall_results.append({
            'Image': img_info['file_name'],
            'GT': gt_label,
            'Rotation': angle,
            'Prediction': verdict,
            'Detections': len(boxes),
            'Confidence': f"{np.mean(scores):.2f}" if len(scores) > 0 else "0.00",
            'Correct': match
        })

plt.suptitle('Complete Rotation Test Matrix (2 Defect + 2 Good)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Summary statistics
results_df = pd.DataFrame(overall_results)

print("\n" + "="*60)
print("ROTATION TEST SUMMARY")
print("="*60)

# Overall accuracy
correct_count = len(results_df[results_df['Correct'] == '✓'])
total_count = len(results_df)
accuracy = (correct_count / total_count) * 100

print(f"\nOverall Accuracy: {correct_count}/{total_count} ({accuracy:.1f}%)")

# By rotation angle
print("\nAccuracy by Rotation Angle:")
for angle in rotation_angles.keys():
    angle_df = results_df[results_df['Rotation'] == angle]
    correct = len(angle_df[angle_df['Correct'] == '✓'])
    total = len(angle_df)
    acc = (correct / total) * 100 if total > 0 else 0
    print(f"  {angle:>4s}: {correct}/{total} ({acc:.1f}%)")

# By image type
print("\nAccuracy by Image Type:")
for label in ['Defect', 'Good']:
    label_df = results_df[results_df['GT'] == label]
    correct = len(label_df[label_df['Correct'] == '✓'])
    total = len(label_df)
    acc = (correct / total) * 100 if total > 0 else 0
    print(f"  {label:>6s}: {correct}/{total} ({acc:.1f}%)")

print("\n" + "="*60)
print("\nDetailed Results Table:")
display(results_df)

print("\n✓ Complete rotation matrix test finished")
